# Imports

In [18]:
import sys, os
is_conda = os.path.exists(os.path.join(sys.prefix, 'conda-meta'))

if not is_conda:
    import findspark 
    findspark.init()

from pyspark.sql import SparkSession
# import pandas as pd
# import numpy as np
# import matplotlib as plt
from datetime import datetime
from pyspark.sql.functions import col, datediff, unix_timestamp

# Lectura de datos

In [11]:
spark = SparkSession.builder.appName("taxis").master("local[*]").getOrCreate()
df = spark.read.csv('./tripdata_2017_01.csv', header=True, inferSchema=True)

In [12]:
df.printSchema()
dfP=df.toPandas()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



# Limpieza de datos

In [13]:
display(dfP)
display(dfP.describe().T)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2017-01-09 11:13:28,2017-01-09 11:25:45,1,3.30,1,N,263,161,1,12.5,0.0,0.5,2.00,0.00,0.3,15.30
1,1,2017-01-09 11:32:27,2017-01-09 11:36:01,1,0.90,1,N,186,234,1,5.0,0.0,0.5,1.45,0.00,0.3,7.25
2,1,2017-01-09 11:38:20,2017-01-09 11:42:05,1,1.10,1,N,164,161,1,5.5,0.0,0.5,1.00,0.00,0.3,7.30
3,1,2017-01-09 11:52:13,2017-01-09 11:57:36,1,1.10,1,N,236,75,1,6.0,0.0,0.5,1.70,0.00,0.3,8.50
4,2,2017-01-01 00:00:00,2017-01-01 00:00:00,1,0.02,2,N,249,234,2,52.0,0.0,0.5,0.00,0.00,0.3,52.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971005,2,2017-01-17 13:16:22,2017-01-17 13:35:14,1,3.43,1,N,264,264,2,14.5,0.0,0.5,0.00,0.00,0.3,15.30
971006,2,2017-01-17 13:16:23,2017-01-17 13:23:01,1,0.78,1,N,161,237,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16
971007,2,2017-01-17 13:16:23,2017-01-17 13:23:51,1,0.95,1,N,237,229,2,6.5,0.0,0.5,0.00,0.00,0.3,7.30
971008,2,2017-01-17 13:16:23,2017-01-17 13:32:45,1,1.68,1,N,170,48,1,11.5,0.0,0.5,2.46,0.00,0.3,14.76


,count,mean,std,min,25%,50%,75%,max
VendorID,971010.0,1.556951,0.496746,1.00,1.00,2.00,2.00,2.00
passenger_count,971010.0,1.685989,1.291705,0.00,1.00,1.00,2.00,9.00
trip_distance,971010.0,3.031285,3.785847,0.00,1.00,1.70,3.25,151.70
RatecodeID,971010.0,1.044729,0.461631,1.00,1.00,1.00,1.00,99.00
PULocationID,971010.0,160.378031,67.938198,1.00,113.00,161.00,231.00,265.00
DOLocationID,971010.0,158.556712,72.276821,1.00,100.00,161.00,233.00,265.00
payment_type,971010.0,1.371974,0.503326,1.00,1.00,1.00,2.00,4.00
fare_amount,971010.0,13.108188,546.736933,-120.00,6.50,9.00,14.00,538579.20
extra,971010.0,0.209993,0.261634,-1.00,0.00,0.00,0.50,55.54
mta_tax,971010.0,0.497214,0.040586,-0.50,0.50,0.50,0.50,0.50


### Elementos extraños en el dataset

Lista de comportamientos extraños en los datos, y por tanto, inválidos a la hora de utilizar datos que deberían ser coherentes basándonos en la información de cada campo proporcionada por la [documentación](https://www1.nyc.gov/assets/tlc/downloads/pdf/data_dictionary_trip_records_yellow.pdf)

* Existen carreras en las que la distancia es 0
* Existen propinas negativas
* "extra" con valores diferentes a 0 (ya que puede no haber extras), 0.5 y 1
* Existen viajes con un precio final negativo
* "MTA_tax" debe valer siempre 0.50. Valores diferentes son erróneos, y por tanto puede que el resto de la información también
    * De forma similar, "Improvement_surcharge" no debe valer menos de 0.30
* Carreras cuya fecha de fin sea igual o anterior a la fecha de inicio
* Existen tarifas con valores negativos. No tiene sentido ya que la tarifa va en función del tiempo y la distancia recorridas
* "Improvement_surcharge" es un valor en desuso, por lo que debería valer en el menor cas 0, no -0.3

### Elementos extraños PERO posibles

* Número de pasajeros es 0. Dado que es un valor que introduce el propio conductor, muy probablemente le de bastante igual introducir bien el valor.
* Un viaje empieza y acaba en la misma zona.



### Limpieza realizada

A partir de los comportamientos observados se ha procedido a eliminar las carreras que cumplen las siguientes condiciones:

- Campo "tip_amount" con valores menor a 0
- Campo "total_amount" con valores menor o igual a 0
- Campo "trip_distance" con valores menor o igual a 0
- Campo "fare_amount" con valores menor o igual a 0
- Campo "extra" con valores diferentes de 0, 0.5 y 1
- Campo "MTA_tax" con valor distinto de 0.5
- Campo "Improvement_surcharge" con valor distinto de 0 o 0.3
- Campo "tpep_dropoff_datetime" es anterior o igual a "tpep_pickup_datetime"

In [32]:
# Convertimos las fechas a timestamp, para que dejen de ser strings a secas
# y guardamos su diferencia para luego tener más fácil el filtrado y otros cálculos

# ----------------------------------------------------------------------------------
# ESTO ES ABSURDAMENTE LENTO, TIENE QUE HABER ALGUNA FORMA MÁS FÁCIL DE HACER ESTO
# ----------------------------------------------------------------------------------
df = df.withColumn(
    "tpep_pickup_timestamp", unix_timestamp(col("tpep_pickup_datetime").cast("timestamp"))
).withColumn(
    "tpep_dropoff_timestamp", unix_timestamp(col("tpep_dropoff_datetime").cast("timestamp"))
).withColumn(
    "time_diff", col("tpep_dropoff_timestamp") - col("tpep_pickup_timestamp")
)

df.createOrReplaceTempView('datosCarreras')
# display(df.toPandas())

In [33]:
datosLimpios = spark.sql("""
    SELECT * FROM datosCarreras WHERE
        tip_amount >= 0 AND
        total_amount > 0 AND
        trip_distance > 0 AND
        fare_amount > 0 AND
        (extra == 0 OR extra == 0.5 OR extra == 1) AND
        mta_tax == 0.5 AND
        improvement_surcharge >= 0 AND
        time_diff > 0
""")
print(datosLimpios.count())
datosLimpios.createOrReplaceTempView('datosCarrerasLimpios')
datosLimpiosP = datosLimpios.toPandas()

961150


In [30]:
display(datosLimpiosP)
display(datosLimpiosP.describe().T)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,time_diff
0,1,1483956808,1483957545,1,3.30,1,N,263,161,1,12.5,0.0,0.5,2.00,0.00,0.3,15.30,737
1,1,1483957947,1483958161,1,0.90,1,N,186,234,1,5.0,0.0,0.5,1.45,0.00,0.3,7.25,214
2,1,1483958300,1483958525,1,1.10,1,N,164,161,1,5.5,0.0,0.5,1.00,0.00,0.3,7.30,225
3,1,1483959133,1483959456,1,1.10,1,N,236,75,1,6.0,0.0,0.5,1.70,0.00,0.3,8.50,323
4,1,1483225202,1483225430,1,0.50,1,N,48,48,2,4.0,0.5,0.5,0.00,0.00,0.3,5.30,228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
961145,2,1484655382,1484656514,1,3.43,1,N,264,264,2,14.5,0.0,0.5,0.00,0.00,0.3,15.30,1132
961146,2,1484655383,1484655781,1,0.78,1,N,161,237,1,6.0,0.0,0.5,1.36,0.00,0.3,8.16,398
961147,2,1484655383,1484655831,1,0.95,1,N,237,229,2,6.5,0.0,0.5,0.00,0.00,0.3,7.30,448
961148,2,1484655383,1484656365,1,1.68,1,N,170,48,1,11.5,0.0,0.5,2.46,0.00,0.3,14.76,982


,count,mean,std,min,25%,50%,75%,max
VendorID,961150.0,1.557943e+00,0.496632,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00
tpep_pickup_datetime,961150.0,1.484164e+09,579104.512405,1.483225e+09,1.483298e+09,1.484470e+09,1.484582e+09,1.485874e+09
tpep_dropoff_datetime,961150.0,1.484165e+09,579089.397652,1.483225e+09,1.483299e+09,1.484471e+09,1.484583e+09,1.485875e+09
passenger_count,961150.0,1.687589e+00,1.293372,0.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00
trip_distance,961150.0,3.016129e+00,3.696633,1.000000e-02,1.000000e+00,1.700000e+00,3.250000e+00,1.391700e+02
RatecodeID,961150.0,1.026247e+00,0.188597,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,6.000000e+00
PULocationID,961150.0,1.603090e+02,67.864522,1.000000e+00,1.130000e+02,1.610000e+02,2.310000e+02,2.650000e+02
DOLocationID,961150.0,1.586092e+02,71.978933,1.000000e+00,1.000000e+02,1.610000e+02,2.320000e+02,2.650000e+02
payment_type,961150.0,1.368578e+00,0.498094,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,4.000000e+00
fare_amount,961150.0,1.233191e+01,10.246143,2.000000e-01,6.500000e+00,9.000000e+00,1.400000e+01,3.500000e+02


## Extracción de información

Ahora que ya hemos limpiado los datos y tenemos entradas coherentes, se puede proceder a extraer información de los mismos. 

La información que se va a extraer es:

* Velocidad media de los taxis en función de la hora.
* Viajes en taxi más comunes
* Registros financieros (propinas, personas, etc.)
    * Timos a turistas
    * Propinas en función de la hora
    * Identificar pasajeros borrachos
* Zonas con poca cobertura



### Velocidad media de los taxis

### Registros financieros

In [ ]:
resultsTip = spark.sql("SELECT * FROM datosCarrerasLimpios where tip_amount >= 100").toPandas()
display(resultsTip)
display(resultsTip.describe().T)

In [ ]:
resultsTolls = spark.sql("SELECT * FROM datosCarrerasLimpios where tolls_amount > 100").toPandas()
display(resultsTolls)
display(resultsTolls.describe().T)

In [ ]:
resultsTimos = spark.sql("SELECT * FROM datosCarrerasLimpios where PULocationID == DOLocationID").toPandas()

In [ ]:
display(resultsTimos.sort_values(by=["trip_distance"],ascending=False).head(20))
display(resultsTimos.describe().T)

### Zonas de poca cobertura

### Velocidad media de los taxis

# Fin

In [ ]:
spark.stop()

IDEAS

propinas / hora

Timos

- Vueltas de mas en misma zona
- Tolls valores raros
- Diferencias exageradas de distancias para pares de datos con mismo origen y destino

Velocidad media de los taxis en función de la hora.

Viajes en taxi más comunes

Registros financieros (propinas, personas, etc.)

Zonas sin cobertura a partir del parámetro Store_and_fwd_flag
